# Analyzing iMessage Conversations

Use this notebook to obtain various statistics about iMessage conversations. Fill out the address book with numbers from your contact book. The names don't matter- they are just used for display. Setting `TARGET` allows you to change what number is analyzed, and `run_export_and_build` determines whether a new export & build cycle is initiated when running the notebook.

> Note that **the statistics and visualizations in this notebook may not be fully accurate**. 
Due to various reasons, there could be problems in the export of your iMessage data, and that would skew the results. Double-check data before using it to support significant insights.
In other words, **take data with grains of salt**.

## Environment Setup

In [1]:
# fill out this with your own phone numbers and names
address_book = {
    '+1234567890': 'name',
}

# the name of the contact we are analyzing
TARGET= "name"
# should we run the export and build the database?
run_export_and_build = False

In [ ]:
# imports
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

print("All libraries imported successfully.")
DB_PATH = "imessage.db"

rev_address_book = {v: k for k, v in address_book.items()}
TARGET_NUMBER = rev_address_book[TARGET]
print(f"Target set to {TARGET} ({TARGET_NUMBER})")
print("Exporting and building: ", "enabled" if run_export_and_build else "disabled")

def parse_date_robust(date_str):
    # Common formats to try
    formats = [
        '%b %d, %Y  %I:%M:%S %p',   # May 16, 2024  1:11:06 PM (double space)
        '%b %d, %Y %I:%M:%S %p',    # May 16, 2024 1:11:06 PM (single space)
        '%B %d, %Y  %I:%M:%S %p',   # December 16, 2024  1:11:06 PM (full month)
        '%B %d, %Y %I:%M:%S %p',    # December 16, 2024 1:11:06 PM
        '%m/%d/%Y %I:%M:%S %p',     # 12/16/2024 1:11:06 PM
        '%Y-%m-%d %H:%M:%S',        # 2024-12-16 13:11:06
    ]
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    # If all formats fail, let pandas infer
    try:
        return pd.to_datetime(date_str, infer_datetime_format=True)
    except:
        return pd.NaT

## Export & Building of Database

In [ ]:
# run export for every current export
if run_export_and_build:
    from export import *
    nums = list(address_book.keys())
    print("Exporting the following numbers:", nums)
    for num in nums:
        export(num, force=True, do_not_clean=True)
    print("All exports completed.")
    clean_exported_dir()
else:
    print("Skipping export and build as per configuration.")

In [ ]:
# build database
if run_export_and_build:
    from database import *
    build_imessage_database("exported", DB_PATH)
else:
    print("Skipping database build as per configuration.")

## Connect to Database

In [ ]:
try:
    conn = sqlite3.connect(DB_PATH)
    print(f"Connected to the database at {DB_PATH}.")
    
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
    print("\nTables in the database:")
    for table in tables['name']:
        print(f"\t- {table}")
    
    # select conversations table and find all unique target numbers
    conversations = pd.read_sql_query("SELECT * FROM conversations", conn)
    unique_numbers = conversations['target_number'].unique()
    print(f"\nUnique target numbers found: {len(unique_numbers)}")
    for number in unique_numbers:
        print(f"\t- {number}")
    print(f"\nSelected target: {TARGET_NUMBER} ({TARGET})")

except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit(1)

## Analysis

### Messages Sent *vs* Recieved

In [ ]:
# query messages of the target number
try:
    # Get a sample of messages
    query = f"""
    SELECT 
        target_number,
        is_from_me
    FROM messages 
    WHERE is_duplicate = FALSE 
    AND target_number = '{TARGET_NUMBER}'
    """

    print(f"Target: {TARGET_NUMBER} ({TARGET}) \n")
    
    sample_df = pd.read_sql_query(query, conn)

    # calculate and print stats
    # message ratio, total sent, total recieved
    sent_count = sample_df[sample_df['is_from_me'] == 1].shape[0]
    received_count = sample_df[sample_df['is_from_me'] == 0].shape[0]
    total_messages = sent_count + received_count

    # message ratio, in sent:recieved, whole numbers
    sent_ratio = 1
    received_ratio = received_count / sent_count if sent_count > 0 else 0
    
    counts = [sent_count, received_count]
    labels = ['Sent', 'Received']
    colors = ['#218aff', "#d8d8d8"]  # Optional: soft color palette

    # visualize using pie chart
    plt.figure(figsize=(7, 7))
    wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)

    autotexts[0].set_color('white')  # Sent text color
    autotexts[1].set_color('black')  # Received text color

    plt.title('Message Distribution: Sent vs Received')
    plt.axis('equal')  # equal aspect ratio ensures the pie chart is circular
    plt.show()

    print(f"\nTotal messages: {total_messages}")
    print(f" no. sent: {sent_count}")
    print(f" no. recieved: {received_count}")
    print(f"(sent:received): {sent_ratio:.2f} : {received_ratio:.4f}")
except Exception as e:
    print(f"Error: {e}")

### Messages over Time

In [ ]:
# query messages of the target number and plot send date over time
debug = False
try:
    query = f"""
    SELECT 
        is_from_me,
        message_date
    FROM messages 
    WHERE is_duplicate = FALSE 
    AND target_number = '{TARGET_NUMBER}'
    """

    print(f"Target: {TARGET_NUMBER} ({TARGET}) \n")
    
    messages = pd.read_sql_query(query, conn)
    
    if debug:
        print("Sample dates from database:")
        print(messages['message_date'].head(10).tolist())
        print("\nUnique date formats found:")
        print(messages['message_date'].str.len().value_counts().head())
    
    def parse_date_robust(date_str):
        # Common formats to try
        formats = [
            '%b %d, %Y  %I:%M:%S %p',   # May 16, 2024  1:11:06 PM (double space)
            '%b %d, %Y %I:%M:%S %p',    # May 16, 2024 1:11:06 PM (single space)
            '%B %d, %Y  %I:%M:%S %p',   # December 16, 2024  1:11:06 PM (full month)
            '%B %d, %Y %I:%M:%S %p',    # December 16, 2024 1:11:06 PM
            '%m/%d/%Y %I:%M:%S %p',     # 12/16/2024 1:11:06 PM
            '%Y-%m-%d %H:%M:%S',        # 2024-12-16 13:11:06
        ]
        
        for fmt in formats:
            try:
                return pd.to_datetime(date_str, format=fmt)
            except:
                continue
        
        # If all formats fail, let pandas infer
        try:
            return pd.to_datetime(date_str, infer_datetime_format=True)
        except:
            if debug:
                print(f"Failed to parse date: {date_str}")
            return pd.NaT
    
    messages['message_date'] = messages['message_date'].apply(parse_date_robust)
    
    # Check for failed parsing
    failed_dates = messages[messages['message_date'].isna()]
    if len(failed_dates) > 0:
        if debug:
            print(f"Warning: {len(failed_dates)} dates failed to parse")
            print("Failed dates sample:")
            print(failed_dates['message_date'].head())
        # Remove failed dates
        messages = messages.dropna(subset=['message_date'])
    
    # sort by date
    messages = messages.sort_values('message_date').reset_index(drop=True)
    
    # create separate dataframes for sent and received messages
    sent_messages = messages[messages['is_from_me'] == 1].copy()
    received_messages = messages[messages['is_from_me'] == 0].copy()
    
    # calculate cumulative counts
    sent_messages['cumulative_sent'] = range(1, len(sent_messages) + 1)
    received_messages['cumulative_received'] = range(1, len(received_messages) + 1)
    
    # create interactive Plotly visualization
    # prepare data for Plotly (long format)
    plot_data = []
    
    # add sent messages
    for i, row in sent_messages.iterrows():
        plot_data.append({
            'date': row['message_date'],
            'count': row['cumulative_sent'],
            'type': 'Sent'
        })
    
    # add received messages
    for i, row in received_messages.iterrows():
        plot_data.append({
            'date': row['message_date'],
            'count': row['cumulative_received'],
            'type': 'Received'
        })
    
    plot_df = pd.DataFrame(plot_data)
    
    # create Plotly line plot
    custom_colors = {
        'Sent': '#218aff',
        'Received': '#d8d8d8'
    }
    
    fig = px.line(plot_df, 
                  x='date', 
                  y='count', 
                  color='type',
                  color_discrete_map=custom_colors,
                  title=f'Cumulative Messages Over Time',
                  labels={
                      'date': 'Date',
                      'count': 'Cumulative Message Count',
                      'type': 'Message Type'
                  })
    
    fig.update_traces(
        line=dict(width=3),
        mode='lines',
        hovertemplate='<b>%{fullData.name}</b><br>' +
                      'Date: %{x}<br>' +
                      'Count: %{y:,}<br>' +
                      '<extra></extra>'   # Custom hover info
    )
    
    fig.update_layout(
        height=800,
        template='plotly_white',
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        
        font=dict(size=13, family="Arial"),
        title=dict(
            x=0.5,
            pad=dict(t=20)
        ),
        
        legend=dict(
            x=0.01,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1
        ),
        
        # Axis styling
        xaxis=dict(
            title=dict(font=dict(size=14)),
            tickfont=dict(size=12),
            gridcolor='rgba(128,128,128,0.2)',
            showline=True,
            linecolor='rgba(128,128,128,0.5)',
            tickmode='linear',
            dtick = "M1"
        ),
        yaxis=dict(
            title=dict(font=dict(size=14)),
            tickfont=dict(size=12),
            gridcolor='rgba(128,128,128,0.2)',
            showline=True,
            linecolor='rgba(128,128,128,0.5)'
        ),
        
        hovermode='x unified',
        
        margin=dict(l=60, r=60, t=100, b=60)
    )
    
    fig.show()
    
    # Print summary statistics
    print(f"\nSummary Statistics:")
    print(f"Total Messages: {len(messages)}")
    print(f"Sent Messages: {len(sent_messages)}")
    print(f"Received Messages: {len(received_messages)}")
    print(f"Date Range: {messages['message_date'].min()} to {messages['message_date'].max()}")

    # expose the cumalative dataframes for further analysis
    SENT_CUMULATIVE = sent_messages['cumulative_sent']
    RECEIVED_CUMULATIVE = received_messages['cumulative_received']
    locals().update({
        'SENT_CUMULATIVE': SENT_CUMULATIVE,
        'RECEIVED_CUMULATIVE': RECEIVED_CUMULATIVE
    })

except Exception as e:
    print(f"Error while plotting messages over time: {e}")

### Message Rate over Time

In [ ]:
# Calculate and plot the rate of messages over time with rolling average
rolling_window = 1 
try:
    # Use the dataframes from the previous step
    if 'sent_messages' in locals() and 'received_messages' in locals() and 'messages' in locals():
        # Calculate daily message rates
        # Resample to daily frequency and count messages per day
        sent_daily = sent_messages.set_index('message_date').resample('D').size()
        received_daily = received_messages.set_index('message_date').resample('D').size()
        
        # Fill missing days with 0
        sent_daily = sent_daily.fillna(0)
        received_daily = received_daily.fillna(0)
        
        # Calculate rolling averages
        sent_rolling = sent_daily.rolling(window=rolling_window, center=True).mean()
        received_rolling = received_daily.rolling(window=rolling_window, center=True).mean()
        
        # Create DataFrame for plotting
        rate_data = []
        
        # Add sent messages rate
        for date, rate in sent_rolling.dropna().items():
            rate_data.append({
                'date': date,
                'rate': rate,
                'type': 'Sent'
            })
        
        # Add received messages rate  
        for date, rate in received_rolling.dropna().items():
            rate_data.append({
                'date': date,
                'rate': rate,
                'type': 'Received'
            })
        
        rate_df = pd.DataFrame(rate_data)
        
        # Custom colors to match your theme
        custom_colors = {
            'Sent': '#218aff',
            'Received': '#d8d8d8'
        }
        
        # Create Plotly line plot
        fig = px.line(rate_df,
                      x='date',
                      y='rate', 
                      color='type',
                      color_discrete_map=custom_colors,
                      title=f'Message Rate Over Time ({rolling_window}-Day Rolling Average)',
                      labels={
                          'date': 'Date',
                          'rate': 'Messages per Day',
                          'type': 'Message Type'
                      })
        
        fig.update_traces(
            line=dict(width=3),
            mode='lines',
            hovertemplate='<b>%{fullData.name}</b><br>' +
                          'Date: %{x}<br>' +
                          'Rate: %{y:.1f} msg/day<br>' +
                          '<extra></extra>'
        )
        
        fig.update_layout(
            height=800,
            template='plotly_white',
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            
            font=dict(size=13, family="Arial"),
            title=dict(
                x=0.5,
                pad=dict(t=20)
            ),
            
            legend=dict(
                x=0.01,
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="rgba(0,0,0,0.2)",
                borderwidth=1
            ),
            
            # Axis styling
            xaxis=dict(
                title=dict(font=dict(size=14)),
                tickfont=dict(size=12),
                gridcolor='rgba(128,128,128,0.2)',
                showline=True,
                linecolor='rgba(128,128,128,0.5)',
                tickmode='linear',
                dtick="M1"  # Monthly ticks to match your style
            ),
            yaxis=dict(
                title=dict(font=dict(size=14)),
                tickfont=dict(size=12),
                gridcolor='rgba(128,128,128,0.2)',
                showline=True,
                linecolor='rgba(128,128,128,0.5)'
            ),
            
            hovermode='x unified',
            margin=dict(l=60, r=60, t=100, b=60)
        )
        
        fig.show()
        
        # Print rate statistics
        print(f"\nMessage Rate Statistics ({rolling_window}-Day Rolling Average):")
        print(f"Peak Sent Rate: {sent_rolling.max():.1f} messages/day")
        print(f"Peak Received Rate: {received_rolling.max():.1f} messages/day")
        print(f"Average Sent Rate: {sent_rolling.mean():.1f} messages/day")
        print(f"Average Received Rate: {received_rolling.mean():.1f} messages/day")
        print(f"Date Range: {rate_df['date'].min()} to {rate_df['date'].max()}")
        
        # Expose the rate dataframes for further analysis
        SENT_RATE = sent_rolling
        RECEIVED_RATE = received_rolling
        locals().update({
            'SENT_RATE': SENT_RATE,
            'RECEIVED_RATE': RECEIVED_RATE,
            'rate_df': rate_df
        })
        
    else:
        print("Message dataframes not found. Make sure to run the cumulative analysis first.")
        
except Exception as e:
    print(f"Error while plotting message rate over time: {e}")

### Characters Sent *vs* Recieved

In [ ]:
# sent vs recieved characters, pie chart
try:
    query = f"""
    SELECT 
        target_number,
        is_from_me,
        length(message_text) AS message_length
    FROM messages 
    WHERE is_duplicate = FALSE 
    AND message_type = 'text'
    AND target_number = '{TARGET_NUMBER}'
    """

    print(f"Target: {TARGET_NUMBER} ({TARGET}) \n")
    
    sample_df = pd.read_sql_query(query, conn)

    # calculate and print stats
    # message ratio, total sent, total recieved
    sent_msg_count = sample_df[sample_df['is_from_me'] == 1].shape[0]
    received__msg_count = sample_df[sample_df['is_from_me'] == 0].shape[0]
    sent_count = sample_df[sample_df['is_from_me'] == 1]['message_length'].sum()
    received_count = sample_df[sample_df['is_from_me'] == 0]['message_length'].sum()
    total_chars = sent_count + received_count

    # message ratio, in sent:recieved, whole numbers
    sent_ratio = 1
    received_ratio = received_count / sent_count if sent_count > 0 else 0
    
    counts = [sent_count, received_count]
    labels = ['Sent', 'Received']
    colors = ['#218aff', "#d8d8d8"]  # Optional: soft color palette

    # visualize using pie chart
    plt.figure(figsize=(7, 7))
    wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)

    autotexts[0].set_color('white')  # Sent text color
    autotexts[1].set_color('black')  # Received text color

    plt.title('Character Distribution: Sent vs Received')
    plt.axis('equal')  # equal aspect ratio ensures the pie chart is circular
    plt.show()

    print(f"\nTotal characters exchanged: {total_chars} chars")
    print(f" no. sent: {sent_count} chars")
    print(f" avg. sent: {sent_count / sent_msg_count: .3f} chars/message")
    print(f" no. recieved: {received_count} chars")
    print(f" avg. recieved: {received_count / received__msg_count:.3f} chars/message")
    print(f"(sent:received): {sent_ratio:.2f} : {received_ratio:.4f} chars")
except Exception as e:
    print(f"Error: {e}")

### Characters over time

In [ ]:
# same thing as sent and recieved, but counting message length instead
try:
    query = f"""
    SELECT 
        is_from_me,
        message_date,
        length(message_text) AS message_length
    FROM messages 
    WHERE is_duplicate = FALSE 
    AND message_type = 'text'
    AND target_number = '{TARGET_NUMBER}'
    """

    print(f"Target: {TARGET_NUMBER} ({TARGET}) \n")
    
    messages = pd.read_sql_query(query, conn)
    

    
    messages['message_date'] = messages['message_date'].apply(parse_date_robust)
    
    # Check for failed parsing
    failed_dates = messages[messages['message_date'].isna()]
    if len(failed_dates) > 0:
        # Remove failed dates
        messages = messages.dropna(subset=['message_date'])
    
    # sort by date
    messages = messages.sort_values('message_date').reset_index(drop=True)
    
    # create separate dataframes for sent and received messages
    sent_messages = messages[messages['is_from_me'] == 1].copy()
    received_messages = messages[messages['is_from_me'] == 0].copy()
    
    # calculate cumulative counts of characters
    sent_messages['cumulative_sent_chars'] = sent_messages['message_length'].cumsum()
    received_messages['cumulative_received_chars'] = received_messages['message_length'].cumsum()
    
    # create interactive Plotly visualization
    # prepare data for Plotly (long format)
    plot_data = []
    
    # add sent messages
    for i, row in sent_messages.iterrows():
        plot_data.append({
            'date': row['message_date'],
            'count': row['cumulative_sent_chars'],
            'type': 'Sent'
        })
    
    # add received messages
    for i, row in received_messages.iterrows():
        plot_data.append({
            'date': row['message_date'],
            'count': row['cumulative_received_chars'],
            'type': 'Received'
        })
    
    plot_df = pd.DataFrame(plot_data)
    
    # create Plotly line plot
    custom_colors = {
        'Sent': '#218aff',
        'Received': '#d8d8d8'
    }
    
    fig = px.line(plot_df, 
                  x='date', 
                  y='count', 
                  color='type',
                  color_discrete_map=custom_colors,
                  title=f'Cumulative Characters Over Time',
                  labels={
                      'date': 'Date',
                      'count': 'Cumulative Character Count',
                      'type': 'Message Type'
                  })
    
    fig.update_traces(
        line=dict(width=3),
        mode='lines',
        hovertemplate='<b>%{fullData.name}</b><br>' +
                      'Date: %{x}<br>' +
                      'Count: %{y:,}<br>' +
                      '<extra></extra>'   # Custom hover info
    )
    
    fig.update_layout(
        height=800,
        template='plotly_white',
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        
        font=dict(size=13, family="Arial"),
        title=dict(
            x=0.5,
            pad=dict(t=20)
        ),
        
        legend=dict(
            x=0.01,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1
        ),
        
        # Axis styling
        xaxis=dict(
            title=dict(font=dict(size=14)),
            tickfont=dict(size=12),
            gridcolor='rgba(128,128,128,0.2)',
            showline=True,
            linecolor='rgba(128,128,128,0.5)',
            tickmode='linear',
            dtick = "M1"
        ),
        yaxis=dict(
            title=dict(font=dict(size=14)),
            tickfont=dict(size=12),
            gridcolor='rgba(128,128,128,0.2)',
            showline=True,
            linecolor='rgba(128,128,128,0.5)'
        ),
        
        hovermode='x unified',
        
        margin=dict(l=60, r=60, t=100, b=60)
    )
    
    fig.show()
    
    # Print summary statistics
    print(f"\nSummary Statistics:")
    print(f"Total Characters Sent: {sent_messages['cumulative_sent_chars'].max():,}")
    print(f"Total Characters Received: {received_messages['cumulative_received_chars'].max():,}")
    print(f"Date Range: {messages['message_date'].min()} to {messages['message_date'].max()}")

    # expose the cumalative dataframes for further analysis
    SENT_CUMULATIVE_CHARS = sent_messages['cumulative_sent_chars']
    RECEIVED_CUMULATIVE_CHARS = received_messages['cumulative_received_chars']
    locals().update({
        'SENT_CUMULATIVE_CHARS': SENT_CUMULATIVE_CHARS,
        'RECEIVED_CUMULATIVE_CHARS': RECEIVED_CUMULATIVE_CHARS
    })

except Exception as e:
    print(f"Error while plotting messages over time: {e}")

### Message Activity

#### Over the Year

In [ ]:
# message dates, graphed ignoring year (in scatterplot)
try:
    query = """
    SELECT 
        is_from_me,
        message_date
    FROM messages
    WHERE message_date IS NOT NULL
    AND is_duplicate = FALSE
    AND target_number = ?
    """
    messages = pd.read_sql_query(query, conn, params=(TARGET_NUMBER,))
    # use robust date parsing to handle potential format issues
    messages['message_date'] = messages['message_date'].apply(parse_date_robust)
    messages['message_date'] = pd.to_datetime(messages['message_date'])
    
    # Create a continuous day-of-year value (1-366)
    messages['day_of_year'] = messages['message_date'].dt.dayofyear
    
    # Create a date string in MM-DD format for hover info
    messages['date_no_year'] = messages['message_date'].dt.strftime('%m-%d')
    
    # Separate sent and received messages
    sent_messages = messages[messages['is_from_me'] == 1].copy()
    received_messages = messages[messages['is_from_me'] == 0].copy()
    
    # Create scatter plot
    fig = go.Figure()
    
    # Add sent messages - using numerical x-values for proper spacing
    pointSize, pointOpacity = 10, 0.01
    fig.add_trace(go.Scatter(
        x=[0] * len(sent_messages),  # Using 0 for Sent
        y=sent_messages['day_of_year'],
        customdata=sent_messages['date_no_year'],
        mode='markers',
        name='Sent',
        marker=dict(size=pointSize, color='#218aff', opacity=pointOpacity),
        hovertemplate='Date: %{customdata}<extra></extra>'
    ))
    
    # Add received messages - using numerical x-values for proper spacing
    fig.add_trace(go.Scatter(
        x=[1] * len(received_messages),  # Using 1 for Received
        y=received_messages['day_of_year'],
        customdata=received_messages['date_no_year'],
        mode='markers',
        name='Received',
        marker=dict(size=pointSize, color='black', opacity=pointOpacity),
        hovertemplate='Date: %{customdata}<extra></extra>'
    ))
    
    # Create month tick labels - positioned at middle of each month
    month_positions = {
        15: 'Jan', 45: 'Feb', 75: 'Mar', 105: 'Apr', 
        135: 'May', 166: 'Jun', 196: 'Jul', 227: 'Aug',
        258: 'Sep', 288: 'Oct', 319: 'Nov', 349: 'Dec'
    }
    
    # Update layout
    fig.update_layout(
        title='Message Activity Over the Year',
        xaxis_title='Message Type',
        yaxis_title='Date (Month)',
        height=600,
        showlegend=True,
        hovermode='closest',
        yaxis=dict(
            tickmode='array',
            tickvals=list(month_positions.keys()),
            ticktext=list(month_positions.values()),
            showgrid=True,
            range=[1, 366]  # Full year range
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=[0, 1],
            ticktext=['Sent', 'Received'],
            showgrid=False,
            range=[-0.5, 1.5]  # Add padding on both sides
        ),
        plot_bgcolor='rgba(240,240,240,0.2)'
    )
    
    fig.show()
except Exception as e:
    print(f"Error: {e}")

#### Over the Day


In [ ]:
# message activity by time of day (in scatterplot with continuous scale)
try:
    query = """
    SELECT 
        is_from_me,
        message_date
    FROM messages
    WHERE message_date IS NOT NULL
    AND is_duplicate = FALSE
    AND target_number = ?
    """
    messages = pd.read_sql_query(query, conn, params=(TARGET_NUMBER,))
    # use robust date parsing to handle potential format issues
    messages['message_date'] = messages['message_date'].apply(parse_date_robust)
    messages['message_date'] = pd.to_datetime(messages['message_date'])
    
    # Extract continuous time of day (hours as decimal)
    # This creates a value between 0.0 and 23.999 representing the exact time
    messages['time_of_day'] = messages['message_date'].dt.hour + \
                             messages['message_date'].dt.minute / 60.0 + \
                             messages['message_date'].dt.second / 3600.0
    
    # Create precise time string for hover info (HH:MM:SS)
    messages['time_str'] = messages['message_date'].dt.strftime('%H:%M:%S')
    
    # Separate sent and received messages
    sent_messages = messages[messages['is_from_me'] == 1].copy()
    received_messages = messages[messages['is_from_me'] == 0].copy()
    
    # Create scatter plot
    fig = go.Figure()
    
    # Add sent messages
    fig.add_trace(go.Scatter(
        x=[0] * len(sent_messages),  # Using 0 for Sent
        y=sent_messages['time_of_day'],
        customdata=sent_messages['time_str'],
        mode='markers',
        name='Sent',
        marker=dict(size=7, color='#218aff', opacity=0.01),
        hovertemplate='Time: %{customdata}<extra></extra>'
    ))
    
    # Add received messages
    fig.add_trace(go.Scatter(
        x=[1] * len(received_messages),  # Using 1 for Received
        y=received_messages['time_of_day'],
        customdata=received_messages['time_str'],
        mode='markers',
        name='Received',
        marker=dict(size=7, color='black', opacity=0.01),
        hovertemplate='Time: %{customdata}<extra></extra>'
    ))
    
    # Create hour labels with AM/PM format for tick marks
    hour_labels = {
        0: '12 AM', 3: '3 AM', 6: '6 AM', 9: '9 AM', 
        12: '12 PM', 15: '3 PM', 18: '6 PM', 21: '9 PM'
    }
    
    # Update layout
    fig.update_layout(
        title='Message Activity by Hour of Day',
        xaxis_title='Message Type',
        yaxis_title='Time of Day',
        height=600,
        showlegend=True,
        hovermode='closest',
        yaxis=dict(
            tickmode='array',
            tickvals=list(hour_labels.keys()),
            ticktext=list(hour_labels.values()),
            showgrid=True,
            range=[-0.5, 24],  # Full day range
            dtick=1  # Minor grid lines every hour
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=[0, 1],
            ticktext=['Sent', 'Received'],
            showgrid=False,
            range=[-0.5, 1.5]  # Add padding on both sides
        ),
        plot_bgcolor='rgba(240,240,240,0.2)'
    )
    
    # Add horizontal grid lines to separate day/night periods
    # Morning (6 AM)
    fig.add_shape(
        type="line",
        x0=-0.5, x1=1.5,
        y0=6, y1=6,
        line=dict(color="rgba(255,165,0,0.2)", width=1.5)
    )
    
    # Noon (12 PM)
    fig.add_shape(
        type="line",
        x0=-0.5, x1=1.5,
        y0=12, y1=12,
        line=dict(color="rgba(255,215,0,0.2)", width=1.5)
    )
    
    # Evening (6 PM)
    fig.add_shape(
        type="line",
        x0=-0.5, x1=1.5,
        y0=18, y1=18,
        line=dict(color="rgba(70,130,180,0.2)", width=1.5)
    )
    
    # Midnight
    fig.add_shape(
        type="line",
        x0=-0.5, x1=1.5,
        y0=0, y1=0,
        line=dict(color="rgba(75,0,130,0.2)", width=1.5)
    )
    
    fig.show()
except Exception as e:
    print(f"Error: {e}")


# Cleanup

In [ ]:
conn.close()
print("Database connection closed.")